In this notebook, we derive the set of functionally unique 3-input gates using the "minterm" approach. 

In [2]:
from sympy.logic import *
from sympy import symbols
import pandas as pd
import itertools
from tqdm import tqdm_notebook as tqdn

### Create list of all possible inputs

The possible combinations of "inputs" to the system are represented by each row of a logic table. Thus, for an n = 3 input system, there are $2^n$ = $2^3$ = 8 possible combinations of inputs:

| Input 1 | Input 2 | Input 3 |
|---------|---------|---------|
|    0    |    0    |    0    |
|    1    |    0    |    0    |
|    1    |    1    |    0    |
|    1    |    0    |    1    |
|    0    |    1    |    0    |
|    0    |    1    |    1    |
|    0    |    0    |    1    |
|    1    |    1    |    1    |


In [3]:
input_combinations = [[input1,input2,input3] for input1 in [0,1] for input2 in [0,1] for input3 in [0,1]]
print('Total input combinations produced by code = '+ str(len(input_combinations)))

Total input combinations produced by code = 8


### Create list of all possible 3 input logic responses

One way to represent a logic response or logic gates is by using "minterms," an approach also known as canonical disjunctive normal form (CDNF). This method characterizes a logical response by a list of the input combinations that produces a "1" output (as opposed to a "0" output). This allows easy conversion of logic tables into a form that lends itself well to comparing logic responses. As an example, the logic response represented by the logic table,

| Input 1 | Input 2 | Input 3 |  Output |
|---------|---------|---------|---------|
|    0    |    0    |    0    |  **0**  |
|    1    |    0    |    0    |  **0**  |
|    1    |    1    |    0    |  **1**  |
|    1    |    0    |    1    |  **1**  |
|    0    |    1    |    0    |  **0**  |
|    0    |    1    |    1    |  **1**  |
|    0    |    0    |    1    |  **0**  |
|    1    |    1    |    1    |  **1**  |

can be succinctly written as a list of its minterms:[[1,1,0], [1,0,1], [0,1,1], [1,1,1]]. Note that the list of all possible minterms is the same as the list of all possible input combinations produced earlier.

Thus creating a list of all possible 3 input logic responses is equivalent to creating a list of all possible combinations of minterms (the list of minterms can be of any length: 0 minterms $\to$ 8 minterms). Since there are m = 8 total minterms and each minterm can be either included or excluded (2 output states possible) when representing a logic response, we expect there to be $2^m$ = $2^8$ = 256 total combinations of minterms (total possible 3 input logic responses).

In [4]:
#create an empty list to store combinations of minterms produced in the for loop 
minterm_combinations = []

#include the no minterm case (where there are no "on", i.e., "1" output responses)
minterm_combinations.append([])

#iterate over all possible lengths of minterm combinations
for combination_length in range(1,len(input_combinations)+1):
    
    #for given length of minterm combinations, list and append all possible combinations of minterms
    for combination in list(itertools.combinations(input_combinations,combination_length)):
        minterm_combinations.append(list(combination))

print('Total minterm combinations (3 input logic responses) produced by code = '+ str(len(minterm_combinations)))

Total minterm combinations (3 input logic responses) produced by code = 256


# Reduce list of all possible 3 input responses to those that are "functionally unique"

We define functional uniqueness between two logic responses if rearranging the input variable names of one logic response cannot reproduce the other logic function. As an example, the 3 input logic responses "Input_1 AND Input_2" and "Input_2 AND Input_3" are considered not unique, since replacing ("mapping") the input variable name "Input_1" in the first logic response with "Input_3" reproduces the second logic response. The logic tables of these responses can be examined for further clarity on this subject: rearranging the header labels of the input columns is equivalent to mapping input variables.

To evaluate the functional uniqueness of our generated list of minterm combinations, we convert minterm combinations to Boolean logic expressions, then write a function to compare two logic expressions. This function is used to do pairwise comparison of all logic functions that have the same number of minterms. Two logic expressions that have different numbers of minterms are by definition not mappable.

Note that there is a sympy.logic function, boolalg.bool_map(), that does something similar to our custom logic comparison function but has some differences in the definition of uniqueness, in particular negation of a variable is allowed when mapping between logic expressions, and thus cannot be used for our purposes.

Also note: all further scripting will be completed using dataframes from the pandas package for organizational purposes. 

In [12]:
# make list corresponding to minterm_combinations that reports whether a minterm combination contains 
# the [0,0,0] minterm. This is for organizational purposes later on.
zeros_minterm = [[0,0,0] in minterm_list for minterm_list in minterm_combinations]

#convert list of all possible minterm combinations into a pandas dataframe
df = pd.DataFrame({'minterms':minterm_combinations,'zeros_minterm_present':zeros_minterm})
df

minterms  zeros_minterm_present
0                                                   []                  False
1                                          [[0, 0, 0]]                   True
2                                          [[0, 0, 1]]                  False
3                                          [[0, 1, 0]]                  False
4                                          [[0, 1, 1]]                  False
5                                          [[1, 0, 0]]                  False
6                                          [[1, 0, 1]]                  False
7                                          [[1, 1, 0]]                  False
8                                          [[1, 1, 1]]                  False
9                               [[0, 0, 0], [0, 0, 1]]                   True
10                              [[0, 0, 0], [0, 1, 0]]                   True
11                              [[0, 0, 0], [0, 1, 1]]                   True
12                              [[0, 0, 0], [1, 0, 0]]                   True
13                              [[0, 0, 0], [1, 0, 1]]                   True
14                              [[0, 0, 0], [1, 1, 0]]                   True
15                              [[0, 0, 0], [1, 1, 1]]                   True
16                              [[0, 0, 1], [0, 1, 0]]                  False
17                              [[0, 0, 1], [0, 1, 1]]                  False
18                              [[0, 0, 1], [1, 0, 0]]                  False
19                              [[0, 0, 1], [1, 0, 1]]                  False
20                              [[0, 0, 1], [1, 1, 0]]                  False
21                              [[0, 0, 1], [1, 1, 1]]                  False
22                              [[0, 1, 0], [0, 1, 1]]                  False
23                              [[0, 1, 0], [1, 0, 0]]                  False
24                              [[0, 1, 0], [1, 0, 1]]                  False
25                              [[0, 1, 0], [1, 1, 0]]                  False
26                              [[0, 1, 0], [1, 1, 1]]                  False
27                              [[0, 1, 1], [1, 0, 0]]                  False
28                              [[0, 1, 1], [1, 0, 1]]                  False
29                              [[0, 1, 1], [1, 1, 0]]                  False
..                                                 ...                    ...
226  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [...                   True
227  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [...                   True
228  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 1], [...                   True
229  [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 0], [...                   True
230  [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 0], [...                   True
231  [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 0], [...                   True
232  [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 1], [...                   True
233  [[0, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 1], [...                   True
234  [[0, 0, 0], [0, 1, 0], [0, 1, 1], [1, 0, 0], [...                   True
235  [[0, 0, 0], [0, 1, 0], [0, 1, 1], [1, 0, 0], [...                   True
236  [[0, 0, 0], [0, 1, 0], [0, 1, 1], [1, 0, 0], [...                   True
237  [[0, 0, 0], [0, 1, 0], [0, 1, 1], [1, 0, 1], [...                   True
238  [[0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 1], [...                   True
239  [[0, 0, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [...                   True
240  [[0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [...                  False
241  [[0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [...                  False
242  [[0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [...                  False
243  [[0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 1], [...                  False
244  [[0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 1], [...                  False
245  [[0, 0, 1], [0, 1, 1], [1, 0, 0], [1, 0, 1], [...              

In [11]:
# create symbolic variables L_1, L_2, L_3 representing the three input ligands
L_1,L_2,L_3 = symbols('L_1 L_2 L_3')
variable_list = [L_1,L_2,L_3]

# add a column to our dataframe for the number of minterms for each logic function
df['num_minterms']=df.apply(lambda row: len(row.minterms),axis=1)

#rearrange columns
df = df[['num_minterms','minterms','zeros_minterm_present']]
df.head(30)

num_minterms                minterms  zeros_minterm_present
0              0                      []                  False
1              1             [[0, 0, 0]]                   True
2              1             [[0, 0, 1]]                  False
3              1             [[0, 1, 0]]                  False
4              1             [[0, 1, 1]]                  False
5              1             [[1, 0, 0]]                  False
6              1             [[1, 0, 1]]                  False
7              1             [[1, 1, 0]]                  False
8              1             [[1, 1, 1]]                  False
9              2  [[0, 0, 0], [0, 0, 1]]                   True
10             2  [[0, 0, 0], [0, 1, 0]]                   True
11             2  [[0, 0, 0], [0, 1, 1]]                   True
12             2  [[0, 0, 0], [1, 0, 0]]                   True
13             2  [[0, 0, 0], [1, 0, 1]]                   True
14             2  [[0, 0, 0], [1, 1, 0]]                   True
15             2  [[0, 0, 0], [1, 1, 1]]                   True
16             2  [[0, 0, 1], [0, 1, 0]]                  False
17             2  [[0, 0, 1], [0, 1, 1]]                  False
18             2  [[0, 0, 1], [1, 0, 0]]                  False
19             2  [[0, 0, 1], [1, 0, 1]]                  False
20             2  [[0, 0, 1], [1, 1, 0]]                  False
21             2  [[0, 0, 1], [1, 1, 1]]                  False
22             2  [[0, 1, 0], [0, 1, 1]]                  False
23             2  [[0, 1, 0], [1, 0, 0]]                  False
24             2  [[0, 1, 0], [1, 0, 1]]                  False
25             2  [[0, 1, 0], [1, 1, 0]]                  False
26             2  [[0, 1, 0], [1, 1, 1]]                  False
27             2  [[0, 1, 1], [1, 0, 0]]                  False
28             2  [[0, 1, 1], [1, 0, 1]]                  False
29             2  [[0, 1, 1], [1, 1, 0]]                  False

### Define a function for comparing boolean statements for uniqueness

In [6]:
def islogicunique(minterms1,minterms2,variables):
    """Tests whether the logic represented by minterms1 and minterms2 are functionally unique or 
    equivalent by checking all possible mappings of variables into minterms1.
    Returns [False,mapping] if a variable mapping exists that reproduces minterms2 from minterms1.
    Returns [True] if no mapping exists."""
    
    # if minterms1 and minterms2 have different numbers of minterms, then no mapping
    # between the two will exist
    if not (len(minterms1) == len(minterms2)):
        return [True]
    
    # generate all permutations (orderings) of variables
    var_permut = itertools.permutations(variables,len(variables))
    
    # generate list of dicts containing each possible permutation mapping
    all_mappings = [{variables[0]:var1,variables[1]:var2,variables[2]:var3} 
                    for (var1,var2,var3) in var_permut]
    
    # iterate through each possible mapping, checking whether it 
    # reproduces minterms1 from minterms2
    for mapping in all_mappings:
        
        # if mapping exists between minterms1 and minterms2, then they are 
        # not functionally unique
        if checkmapping(minterms1,minterms2,variables,mapping):
            return [False,mapping]
        
    return [True]

In [7]:
def checkmapping(minterms1,minterms2,variables,mapping):
    """Checks whether a given mapping of variables into minterms1
    reproduces minterms2, returns True if mapping works"""
    
    # generate ordered list of variable mapping for re-mapping minterms1 
    var_mapping = [mapping[variables[0]],mapping[variables[1]],mapping[variables[2]]]

    # produce boolean logic statement representing minterms1 re-mapped according to mapping
    minterms1_mapped_boolean_statement = SOPform(var_mapping,minterms1)

    # compare minterms1_mapped_boolean_statement to minterms2 to see if they represent
    # equivalent logic
    minterm_eq_result = is_minterm_eq(minterms2,minterms1_mapped_boolean_statement)
    
    if minterm_eq_result:
        
        # if minterms1_mapped_boolean_statement and minterms2 are equivalent, then mapping
        # accurately describes how to reproduce the logic of minterms2 from minterms1
        return True
    
    return False

In [8]:
def is_minterm_eq(minterms,boolean_statement):
    """Compares a list of minterms to a boolean statement,
    returns False if they are different, True if they are equivalent"""
    
    # iterate through each minterm
    for minterm in minterms:
        
        # substitute values from minterm into variables in boolean_statement
        sub_dict = {L_1:minterm[0],L_2:minterm[1],L_3:minterm[2]}
        
        
        if not boolean_statement.subs(sub_dict):
            
            # if statement does not return True, then boolean_statement does not
            # contain this minterm and minterms is not equivalent to boolean_statement
            return False

    return True
        

## Use islogicunique() to do pairwise comparison of all logic functions with same number of minterms

In [9]:
# create empty list for storing results of logic function comparison
mapping = []

# iterate through all logic functions in df
for df_index1,logic1 in tqdn(df.iterrows()):
    
    # generate a reduced dataframe containing all logic functions with the 
    # same number of minterms as logic1, but not including logic1 itself
    logic1_num_minterms = logic1.num_minterms
    df_logic2 = df.query('num_minterms == @logic1_num_minterms').drop(df_index1,'index')
    
    # if there are no logic functions besides logic1 with logic1_num_minterms, then
    # logic1 is unique
    if df_logic2.empty:
        mapping.append('UNIQUE')
        continue
    
    # iterate through df_logic2, comparing each logic2 function to logic1
    for sub_index,(df_index2,logic2) in enumerate(df_logic2.iterrows()):
        
        # test whether logic1 and logic2 are unique
        unique_result = islogicunique(logic1.minterms,logic2.minterms,variable_list)
        
        # if logic1 and logic2 are not unique and logic1 is not the first logic encountered
        # of a functionally equivalent set, store which logic function logic1 maps to and 
        # the mapping. This is to ensure 1 logic function of a functionally equivalent set of 
        # logic functions is stored.
        if df_index1>df_index2 and not unique_result[0]:
            mapping.append((df_index2,unique_result[1]))
            break
            
        # if logic1 has been compared to all logic2's, then logic 1 is unique.
        elif sub_index == len(df_logic2)-1:
            mapping.append('UNIQUE')


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [10]:
# add a column to our dataframe reporting whether each logic function is unique, 
# and how it maps to another logic function if it is not unique
df = df.assign(mapping=mapping)

In [11]:
# select combinations of minterms that are labeled unique
df_uniques = df.loc[df['mapping']=='UNIQUE']
print('The total number of functionally unique gates determined by code = '+ str(len(df_uniques)))
df_uniques

The total number of functionally unique gates determined by code = 80


num_minterms                                           minterms  \
0               0                                                 []   
1               1                                        [[0, 0, 0]]   
2               1                                        [[0, 0, 1]]   
4               1                                        [[0, 1, 1]]   
8               1                                        [[1, 1, 1]]   
9               2                             [[0, 0, 0], [0, 0, 1]]   
11              2                             [[0, 0, 0], [0, 1, 1]]   
15              2                             [[0, 0, 0], [1, 1, 1]]   
16              2                             [[0, 0, 1], [0, 1, 0]]   
17              2                             [[0, 0, 1], [0, 1, 1]]   
20              2                             [[0, 0, 1], [1, 1, 0]]   
21              2                             [[0, 0, 1], [1, 1, 1]]   
28              2                             [[0, 1, 1], [1, 0, 1]]   
30              2                             [[0, 1, 1], [1, 1, 1]]   
37              3                  [[0, 0, 0], [0, 0, 1], [0, 1, 0]]   
38              3                  [[0, 0, 0], [0, 0, 1], [0, 1, 1]]   
41              3                  [[0, 0, 0], [0, 0, 1], [1, 1, 0]]   
42              3                  [[0, 0, 0], [0, 0, 1], [1, 1, 1]]   
49              3                  [[0, 0, 0], [0, 1, 1], [1, 0, 1]]   
51              3                  [[0, 0, 0], [0, 1, 1], [1, 1, 1]]   
58              3                  [[0, 0, 1], [0, 1, 0], [0, 1, 1]]   
59              3                  [[0, 0, 1], [0, 1, 0], [1, 0, 0]]   
60              3                  [[0, 0, 1], [0, 1, 0], [1, 0, 1]]   
62              3                  [[0, 0, 1], [0, 1, 0], [1, 1, 1]]   
64              3                  [[0, 0, 1], [0, 1, 1], [1, 0, 1]]   
65              3                  [[0, 0, 1], [0, 1, 1], [1, 1, 0]]   
66              3                  [[0, 0, 1], [0, 1, 1], [1, 1, 1]]   
72              3                  [[0, 0, 1], [1, 1, 0], [1, 1, 1]]   
86              3                  [[0, 1, 1], [1, 0, 1], [1, 1, 0]]   
87              3                  [[0, 1, 1], [1, 0, 1], [1, 1, 1]]   
..            ...                                                ...   
163             5  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
164             5  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
166             5  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
169             5  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [...   
170             5  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 1], [...   
171             5  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 1], [...   
176             5  [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 1], [...   
177             5  [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 1], [...   
178             5  [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 1, 0], [...   
196             5  [[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0], [...   
198             5  [[0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [...   
200             5  [[0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [...   
201             5  [[0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 1], [...   
202             5  [[0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 1], [...   
207             5  [[0, 0, 1], [0, 1, 0], [1, 0, 1], [1, 1, 0], [...   
211             5  [[0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0], [...   
219             6  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
221             6  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
222             6  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
223             6  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
228             6  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 1], [...   
232             6  [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 1], [...   
240             6  [[0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [...   
241             6  [[0, 0, 1], [0, 1, 0], [0,

In [12]:
# select unique combinations of minterms that contain the [0,0,0] minterm
df_uniques_zeros_on = df_uniques.query('zeros_minterm_present')
print('The total number of functionally unique gates that include the [0,0,0] minterm determined by code = '+ str(len(df_uniques_zeros_on)))
df_uniques_zeros_on

The total number of functionally unique gates that include the [0,0,0] minterm determined by code = 40


num_minterms                                           minterms  \
1               1                                        [[0, 0, 0]]   
9               2                             [[0, 0, 0], [0, 0, 1]]   
11              2                             [[0, 0, 0], [0, 1, 1]]   
15              2                             [[0, 0, 0], [1, 1, 1]]   
37              3                  [[0, 0, 0], [0, 0, 1], [0, 1, 0]]   
38              3                  [[0, 0, 0], [0, 0, 1], [0, 1, 1]]   
41              3                  [[0, 0, 0], [0, 0, 1], [1, 1, 0]]   
42              3                  [[0, 0, 0], [0, 0, 1], [1, 1, 1]]   
49              3                  [[0, 0, 0], [0, 1, 1], [1, 0, 1]]   
51              3                  [[0, 0, 0], [0, 1, 1], [1, 1, 1]]   
93              4       [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1]]   
94              4       [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0]]   
95              4       [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 1]]   
97              4       [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 1, 1]]   
99              4       [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 1]]   
100             4       [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 1, 0]]   
101             4       [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 1, 1]]   
107             4       [[0, 0, 0], [0, 0, 1], [1, 1, 0], [1, 1, 1]]   
121             4       [[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0]]   
122             4       [[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]]   
163             5  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
164             5  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
166             5  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
169             5  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [...   
170             5  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 1], [...   
171             5  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 1], [...   
176             5  [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 1], [...   
177             5  [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 1], [...   
178             5  [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 1, 0], [...   
196             5  [[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0], [...   
219             6  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
221             6  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
222             6  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
223             6  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
228             6  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 1], [...   
232             6  [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 1], [...   
247             7  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
248             7  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
250             7  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   
255             8  [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [...   

     zeros_minterm_present mapping  
1                     True  UNIQUE  
9                     True  UNIQUE  
11                    True  UNIQUE  
15                    True  UNIQUE  
37                    True  UNIQUE  
38                    True  UNIQUE  
41                    True  UNIQUE  
42                    True  UNIQUE  
49                    True  UNIQUE  
51                    True  UNIQUE  
93                    True  UNIQUE  
94                    True  UNIQUE  
95                    True  UNIQUE  
97                    True  UNIQUE  
99                    True  UNIQUE  
100                   True  UNIQUE  
101                   True  UNIQUE  
107                   True  UNIQUE  
121                   True  UNIQUE  
122                   True  UNIQUE  
163                   True  UNIQUE  
164                   True  UNIQUE  
166                   True  UNIQUE  
169                   True  UNIQUE  
170                   True  UNIQUE  
171                   True  UNIQUE  
176                   True  UNIQUE  
177             

In [14]:
df_uniques.to_csv('Functionally_unique_3input_logic_all.csv')
df_uniques_zeros_on.to_csv('Functionally_unique_3input_logic_with_zeros_minterm.csv')